In [68]:
import pandas as pd
import geopandas as gp
import numpy as np 
import os
import fiona
from statistics import mean, median
import string
import re
from collections import Counter
from matplotlib.lines import Line2D

# Voting Checking

Did the vote checking visually on each of the notebooks and there are votes adjusted in Monroe and Seminole Counties to match official totals.

These was the source: https://results.elections.myflorida.com/Index.asp?ElectionDate=11/3/2020&DATAMODE=  

# README Generator

Note: The below code just creates the starting version of the README, it is edited further manually.

In [70]:
'''Following Docstring Convention: https://www.python.org/dev/peps/pep-0257/'''

import pandas as pd
import os
#THESE SETTINGS ESSENTIAL TO HAVE THE FIELDS TABLE SHOW UP CORRECTLY in the readme
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)

'''The functions below support the creation of the final ERJ geodataframe, shapefile export, 
and readme creation/export.
The user must create a fields dictionary to insert into the readme template below.
'''

def select_cols(df, prefix):
    '''select_cols() enables user to select colums by prefix substring,
    so for instance can grab all president columns - enables easy
    re-ordering for the final erj file
    '''
    list = []
    for col in df:
        if prefix in col:
            list.append(col)
    return list

def format_erj_cols(location_desc_cols, election_columns, geo_col):
    col_formatted = location_desc_cols + election_columns + geo_col
    return col_formatted

def create_erj_shp(gdf, shp_name):
    '''In cases where separate election files, run once, then run "gdf.to_file()" 
    separately for each file going to the same directory.
    '''
    os.mkdir('./'+shp_name)
    gdf.to_file('./'+shp_name+'/'+shp_name+'.shp')
    print(shp_name, 'shapefile created.')

def create_fields_table(race_field_header_0, fields_dict_0, 
                        add_race_field_header_1 = '', fields_dict_1 = {}, 
                        add_race_field_header_2 = '', fields_dict_2 = {}, 
                        add_race_field_header_3 = '', fields_dict_3 = {}):
    '''Fields table used in readme
    race_field_header_0: include asterisks "***text***" and label first set of fields
    fields_dict_0: the default dictionary for the primary file (statewide)
    add_race_field_header_1: include asterisks to draw attention to section - ex: "***additional_race_file_name_fields***"
    fields_dict_1: additional fields to go under add_race_field_header_1 header
    add_race_field_header_2 and _3: same use as add_race_field_header_1 - include as needed
    fields_dict_2 and _3: same use as fields_dict_1 - include as needed associated with corresponding add_race_field_header section
    '''
    fields_table_data = {'Field Name': [''] + list(fields_dict_0.keys()) +
                         ['',''] + list(fields_dict_1.keys()) +
                         ['',''] + list(fields_dict_2.keys()) +
                         ['',''] + list(fields_dict_3.keys()),
                         'Description': [race_field_header_0] + list(fields_dict_0.values()) + 
                         ['',add_race_field_header_1] + list(fields_dict_1.values()) + 
                         ['',add_race_field_header_2] + list(fields_dict_2.values()) +
                         ['',add_race_field_header_3] + list(fields_dict_3.values())}
    fields_table = pd.DataFrame(fields_table_data)
    return fields_table

def erj_readme_template(stateabrv, state, year, election_type, additional_races, retrieval_date, vest_file_link, 
                        raw_data_source, state_erj_repo, office_codes, 
                        race_field_header_0, fields_dict_0, add_race_field_header_1 = '', fields_dict_1 = {}, add_race_field_header_2 = '', fields_dict_2 = {}, 
                        add_race_field_header_3 = '', fields_dict_3 = {},
                       additional_notes=' '):
    '''erj_readme_template variable explanations:
    fields_dict = used to create the fields table for the non-standardized/race fields fields. 
                Key is the field/value is the field description
    stateabrv = two character state abbreviation capitalized, 
    state = state name, first letter capitalized, 
    year = election year (XXXX), 
    election_type = general, primary, special or runoff, 
    additional_races = the races that RDH added to the original vest file (not in VEST's og file), 
    retrieval_date = date RDH retrieved VEST file, 
    vest_file_link = link to dataverse page for VEST's precinct boundary and election results file, 
    raw_data_source = site description and link, 
    state_erj_repo = link to erj github repository for given state
    office_codes = codes used broken off of field names for easy viewing. 
        For SU/SL/CON, include ##, so SU## for office code
    race_field_header_0: include asterisks "***text***" and label first set of fields
    fields_dict_0: the default dictionary for the primary file (statewide)
    add_race_field_header_1: include asterisks to draw attention to section - ex: "***additional_race_file_name_fields***"
    fields_dict_1: additional fields to go under add_race_field_header_1 header
    add_race_field_header_2 and _3: same use as add_race_field_header_1 - include as needed
    fields_dict_2 and _3: same use as fields_dict_1 - include as needed associated with corresponding add_race_field_header section
    additional_notes = default set to empty, but fill in with string where applicable.
    '''
    #Generalized readme text
    readme_p1 = '''{year} {stateabrv} {election_type} election results
## RDH Date Retrieval
{retrieval_date}
## Sources
The RDH retrieved the VEST {year} {election_type} precinct boundary and election results shapefile from [VEST's Harvard Dataverse]({vest_file_link})
The RDH retrieved raw {year} {election_type} election results from {raw_data_source}
## Notes on Field Names (adapted from VEST):
Columns reporting votes generally follow the pattern: 
The first character is G for a general election, P for a primary, S for a special, and R for a runoff.
Characters 2 and 3 are the year of the election.*
Characters 4-6 represent the office type (see list below).
Character 7 represents the party of the candidate.
Characters 8-10 are the first three letters of the candidate's last name.
One example is:
G16PREDCLI
To fit within the GIS 10 character limit for field names, the naming convention is slightly different for the State Legislature and 
US House of Representatives. All fields are listed below with definitions.
Office Codes Used:
{office_codes}
## Fields:
'''.format(stateabrv= stateabrv, state= state, year=year, election_type=election_type, additional_races=additional_races,retrieval_date=retrieval_date, vest_file_link=vest_file_link, raw_data_source=raw_data_source, state_erj_repo=state_erj_repo, office_codes=office_codes)

    fields_table = create_fields_table(race_field_header_0, fields_dict_0, add_race_field_header_1, fields_dict_1, add_race_field_header_2, fields_dict_2, 
                        add_race_field_header_3, fields_dict_3).to_string(index=False)
    readme_p2 = str(fields_table)

    readme_p3 = '''
## Processing Steps
    
The RDH joined additional election results to VEST's existing precinct shapefile, including {additional_races} using Python.
For more information on the processing completed, visit our [Github repository]({state_erj_repo}) for Election Result Joins (ERJ) for {state}.
Where possible, the RDH validated the election results we processed against VEST's election results. For additional races the RDH manually checked state totals. For more information on this comparison, please see our processing on Github ({state_erj_repo}).
## Additional Notes
{additional_notes}
Please contact info@redistrictingdatahub.org for more information.
'''.format(stateabrv=stateabrv, state=state, year=year, election_type=election_type, additional_races=additional_races, state_erj_repo=state_erj_repo, office_codes=office_codes, additional_notes = additional_notes)
    
    full_readme = str(readme_p1)+str(readme_p2)+str(readme_p3)
    return full_readme

def export_readme(readme_name, state, election_type, full_readme_text):
    '''readme_name must include file path to readme within erj folder
    ex: 
    readme_name = './az_gen_20_prec/readme.txt'
    '''
    with open(readme_name, 'x') as tf:
        tf.write(full_readme_text)
    print(state, election_type, " readme moved to folder")

In [71]:
stateabrv = "FL"
state = "Florida"
year = "2020"
election_type = "general"
additional_races = "State Supreme Court Justice, Amendments, U.S. House, State House and State Senate"
retrieval_date = "11/08/2021\n"
upload_date = "02/11/2022\n" 
vest_file_link = "https://dataverse.harvard.edu/file.xhtml?fileId=4938250&version=32.0"
raw_data_source = "Election results come from the Florida Division of Elections (https://dos.myflorida.com/elections/data-statistics/elections-data/precinct-level-election-results/) and precinct shapefiles from VEST's Florida 2020 file.\n" 
state_erj_repo = "https://github.com/nonpartisan-redistricting-datahub/erj-fl"
                
additional_notes = '''
~all files~

'''


In [102]:
import os

holder = pd.read_csv("./cand_dict.csv")
fields_dict = dict(zip(holder["Candidate"],holder["Column"]))

In [103]:
office_codes = {}

In [104]:
full_readme = erj_readme_template(fields_dict, stateabrv, state, year, election_type, additional_races, retrieval_date, upload_date, vest_file_link, 
                        raw_data_source, state_erj_repo, office_codes, additional_notes)

In [105]:
race_field_header_0 = ""
fields_dict_0 = {}

In [106]:
full_readme = erj_readme_template(stateabrv, state, year, election_type, additional_races, retrieval_date, vest_file_link, 
                        raw_data_source, state_erj_repo, office_codes, 
                        race_field_header_0, fields_dict, add_race_field_header_1 = '', fields_dict_1 = {}, add_race_field_header_2 = '', fields_dict_2 = {}, 
                        add_race_field_header_3 = '', fields_dict_3 = {},
                       additional_notes=' ')

In [108]:
export_readme("./fl_gen_20_prec/README.txt", "Florida", "general", full_readme)

Florida general  readme moved to folder


## Zip the final file

In [109]:
from zipfile import ZipFile

# more fine-grained control over ZIP files
with ZipFile("fl_gen_20_prec.zip", "w") as newzip:
    for val in os.listdir("./fl_gen_20_prec/"):
        newzip.write("./fl_gen_20_prec/"+val)